## Query Decomposition: Avoid semantic delutions in the queries
 * "Semantic dilutions" refer to the loss of meaning or relevance when chunks of text, intended to represent information for retrieval, become too large or lack semantic cohesion, leading to less accurate and useful results.
 * Semantic dilutions can be mitigated on the indexed documents side by applying optimized chunking strategies per each use case.
 * Query texts can also have semantic delutions when multiple contexts or topics are combined. This can be improved by reforming the queries.

This notebook will demonstrate "query decomposition" strategy to improve the search quality against a knowledge base. 

![Query Decomposition](https://raw.githubusercontent.com/aws-samples/langgraph-agents-with-amazon-bedrock/refs/heads/main/assets/lab3_2.png "https://github.com/aws-samples/langgraph-agents-with-amazon-bedrock/tree/main/Lab_3")

#### Prerequisites

In [1]:

import importlib
import advanced_rag_utils

# Reload module
importlib.reload(advanced_rag_utils)

# Re-import all functions
from advanced_rag_utils import *

from datetime import datetime, timedelta, UTC

notebook_start_time = datetime.now(UTC)
suppress_warnings()


In [2]:
# Load variables from the JSON file
import json
with open("../variables.json", "r") as f:
    variables = json.load(f)

variables

{'accountNumber': '977018081517',
 'regionName': 'us-west-2',
 'collectionArn': 'arn:aws:aoss:us-west-2:977018081517:collection/x6j41w4wldt22s0mc4oa',
 'collectionId': 'x6j41w4wldt22s0mc4oa',
 'vectorIndexName': 'ws-index-',
 'bedrockExecutionRoleArn': 'arn:aws:iam::977018081517:role/advanced-rag-workshop-bedrock_execution_role-us-west-2',
 's3Bucket': '977018081517-us-west-2-advanced-rag-workshop',
 'kbFixedChunk': 'WFJRA66ARJ',
 'kbSemanticChunk': 'XRQUIKPQAB',
 'kbHierarchicalChunk': 'DBCDV6VOVB',
 'kbCustomChunk': 'SVUWGDQTQZ',
 'sagemakerLLMEndpoint': 'endpoint-llama-3-2-3b-instruct-2025-05-01-17-04-39'}

In [3]:
# Get KB configuration
kb_config = get_kb_config(
    kb_id=variables["kbSemanticChunk"],
    account_number=variables['accountNumber'],
    region_name=variables['regionName'],
    number_of_results=5
)

### How Model Size Affects Table Interpretation  

When querying Amazon's Operating Income for 2022, **smaller models (Nova Lite, Llama 3B)** tend to pick the **"At Prior Year Rates"** value (\$11,387), while **larger models (Nova Pro)** correctly select the **"As Reported"** value (\$12,248).

#### Possible Reasons:
- **Table Parsing Limitations:** Smaller models may not accurately align column headers to values.  
- **Context Misinterpretation:** They might default to the last numerical column or fail to strongly associate **"As Reported"** with the correct column.  
- **Stronger Reasoning in Larger Models:** Nova Pro better understands structured data, leading to more accurate retrieval.  

![Image](./operating_income.png)

### Basic RAG query
The query retrieves only one of the topics mentioned. It is because of the mix of two queries dilluted the second topic, which is called "semantic dillution."
In real world scenarios, it is required to pull multiple chunks with different contexts, for example:
* How Amazon's net income increased from 2018 to 2024?
* What is difference between RAG and text-to-SQL?

In [4]:
# WITHOUT QUERY DECOMPOSITION

# Query example
query = "What was Amazon's Operating Income as reported for the fiscal year ending December 31, 2022? What is text-to-SQL? How did text-to-SQL contribute to Amazons earnings, if any?"

# Perform basic RAG without query decomposition
response = retrieve_and_generate_basic(
    query=query, 
    kb_id=kb_config["kb_id"], 
    model_id=kb_config["model_id"], 
    generation_configuration=kb_config["generation_configuration"],
    number_of_results=kb_config["number_of_results"],
    region_name=variables["regionName"]
)

# Display the results
display_rag_results(response)

----------------- Answer ---------------------
Amazon's Operating Income as reported for the fiscal year ending December 31, 2022, was $11,387 million. Text-to-SQL is a technology that allows users to interact with databases using natural language queries. However, the search results do not provide information on how text-to-SQL contributed to Amazon's earnings.

----------------- Citations ------------------
{
  "ResponseMetadata": {
    "RequestId": "ab6b64c0-1368-4d40-a536-272fa4763e0f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 01 May 2025 17:13:27 GMT",
      "content-type": "application/json",
      "content-length": "6196",
      "connection": "keep-alive",
      "x-amzn-requestid": "ab6b64c0-1368-4d40-a536-272fa4763e0f"
    },
    "RetryAttempts": 0
  },
  "citations": [
    {
      "generatedResponsePart": {
        "textResponsePart": {
          "span": {
            "end": 103,
            "start": 0
          },
          "text": "Amazon's Operati

### Use Amazon Bedrock API feature to decompose a query.
ReteriveAndGenerate API support built-in query decompose feature.

In [5]:
# WITH QUERY DECOMPOSITION

# Use Nova Pro model for better table interpretation
nova_pro_model_id = f"arn:aws:bedrock:us-west-2:{variables['accountNumber']}:inference-profile/us.amazon.nova-pro-v1:0"

# Query example (same as before)
query = "What was Amazon's Operating Income as reported for the fiscal year ending December 31, 2022? What is text-to-SQL? How did text-to-SQL contribute to Amazons earnings, if any?"

# Perform RAG with query decomposition
response = retrieve_and_generate_with_decomposition(
    query=query, 
    kb_id=kb_config["kb_id"], 
    model_id=nova_pro_model_id, 
    generation_configuration=kb_config["generation_configuration"],
    number_of_results=kb_config["number_of_results"],
    region_name=variables["regionName"]
)

# Display the results
display_rag_results(response)

----------------- Answer ---------------------
Amazon's Operating Income for the fiscal year ending December 31, 2022, was $12,248 million.

----------------- Citations ------------------
{
  "ResponseMetadata": {
    "RequestId": "be68000a-ece1-4ebb-bc4d-7a9e90eb34b5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 01 May 2025 17:13:29 GMT",
      "content-type": "application/json",
      "content-length": "3804",
      "connection": "keep-alive",
      "x-amzn-requestid": "be68000a-ece1-4ebb-bc4d-7a9e90eb34b5"
    },
    "RetryAttempts": 0
  },
  "citations": [
    {
      "generatedResponsePart": {
        "textResponsePart": {
          "span": {
            "end": 91,
            "start": 0
          },
          "text": "Amazon's Operating Income for the fiscal year ending December 31, 2022, was $12,248 million"
        }
      },
      "retrievedReferences": [
        {
          "content": {
            "text": "Dollar is as follows (in millions): Year Ende

## Query Decomposition with Agentic RAG using SageMaker and LangChain

#### Prerequisites

In [6]:
# Set up LangChain components
llm, retriever = setup_langchain_components(
    endpoint_name=variables["sagemakerLLMEndpoint"],
    kb_id=variables["kbSemanticChunk"],
    region_name=variables["regionName"],
    number_of_results=5
)

#### Test a complex query with plain Q&A chain

In [7]:
# Create a simple QA chain
qa_chain = create_qa_chain(retriever, llm)

In [8]:
# Query example
query = "What was the Operating Income of Amazon As Reported for the Year Ending December 31, 2022? What is text-to-SQL? How did text-to-SQL contribute to Amazons earnings, if any?"

# Invoke the QA chain
answer = qa_chain.invoke(query)

print("Answer:", answer)

Answer:  What was the Operating Income of Amazon As Reported for the Year Ending December 31, 2022?
             The Operating Income of Amazon As Reported for the Year Ending December 31, 2022 is $12,248 million.

             What is text-to-SQL?
             I don't know about it. The context provided does not mention text-to-SQL.

             How did text-to-SQL contribute to Amazons earnings, if any?
             I don't know about it. The context provided does not mention text-to-SQL. Therefore, I cannot provide any information about how it contributed to Amazon's earnings.


### Query Decomposition using Agentic RAG with LangChain

In [9]:
# Create a custom output parser
custom_parser = create_output_parser()

In [10]:
# Set up agentic RAG with query decomposition
agent = setup_agentic_rag(retriever, llm)

# Query example
query = "What was Amazon's Operating Income as reported for the fiscal year ending December 31, 2022? What is text-to-SQL? How did text-to-SQL contribute to Amazons earnings, if any?"

# Run the agent
result = agent.run(query)
print(result)

/home/sagemaker-user/simplified-labs/sample-advanced-rag-using-bedrock-and-sagemaker/Lab 1. Build your RAG Application/advanced_rag_utils.py:2768: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/tmp/ipykernel_1783/2443662035.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res



> Entering new AgentExecutor chain...
 I need to find the answer to the first part of the question, which is Amazon's Operating Income for the fiscal year ending December 31, 2022. I can use the SearchFinancialStatements tool to find this information. For the second part of the question, I need to find out what text-to-SQL is and how it contributed to Amazon's earnings. I can use the SearchTechnologyDocuments tool to find this information.
Action: SearchFinancialStatements
Action Input: Amazon's Operating Income for the fiscal year ending December 31, 2022
Observation: ['Dollar is as follows (in millions): Year Ended December 31, 2022 Year Ended December 31, 2023 \n \n  \n \n As Reported \n \n Exchange Rate \n \n Effect (1) \n \n At Prior Year \n \n Rates (2) As \n \n Reported \n \n Exchange Rate \n \n Effect (1) \n \n At Prior Year \n \n Rates (2) \n \n Net sales $ 513,983 $ 15,495 $ 529,478 $ 574,785 $ 71 $ 574,856 Operating expenses 501,735 16,356 518,091 537,933 531 538,464 Opera